In [188]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
time.sleep(0.2)

# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1.

### Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

In [189]:
URL_habr = 'http://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг'] 
PAGES = 10

In [190]:
# посты с 10 первых страниц 
def get_posts(URL, pages):
    blocks = []      
    
    for page in range(1,pages+1):
        url = URL+'page'+str(page)
        req = requests.get(url)
        time.sleep(0.3)
        
        soup = BeautifulSoup(req.text, 'html.parser')
        articles  = soup.find_all('article', class_='post')
        
        blocks.append(articles)

    return blocks

post_blocks = get_posts(URL_habr, PAGES)

In [191]:
# преобразуем информацию из постов в список словарей 

def create_dictionary(blocks, pages):
    
    posts_total = []
    posts_per_page = []
    
    # каждая страница формирует свой список
    for page in range(len(post_blocks)-1):
        
        # один пост это один словарь, все посты с одной страницы объединены в список 
        for body in post_blocks[page]:
            post_dict = {}      
        
            post_dict['link'] = body.find('a', class_='post__title_link').get('href')
            post_dict['title'] =  body.find('h2', class_='post__title').text
        
            # сразу сделаем весь текст с маленькой буквы
            post_dict['text'] = body.find('div', class_='post__body post__body_crop ').text.lower()
        
            post_dict['date'] = body.find('span', class_='post__time').text
        
            posts_per_page.append(post_dict) 
            
        posts_total.append(posts_per_page) 
        
    return posts_total

post_dict = create_dictionary(post_blocks, PAGES) 

In [192]:
post_dict[0][0]

{'link': 'https://habr.com/ru/company/digital-ecosystems/blog/514860/',
 'title': '\nДайджест IT-событий первой половины сентября\n',
 'text': '\n\n\n\r\nпора отпусков подходит к концу, и мы возвращаемся к своим регулярным обзорам мероприятий, которые помогают программистам развиваться и в нерабочее время. первый учебный месяц начинается знаково – обилием обучающих курсов по разработке и другим дисциплинам. специалисты по кибербезопаности также активно обсуждают старые и новые проблемы, а крупные конференции мало-помалу находят дорогу обратно в оффлайн.\n\nчитать дальше →\n',
 'date': 'сегодня в 14:13'}

In [193]:
# получили ссылки например для всех постов, которые подгрузились

#for i in range(len(post_dict)):
#    for j in range(len(post_dict[i])):
#        print(post_dict[i][j]['link'] )

In [194]:
# фильтрация
# если искомое слово есть в текст превью, то запишем данные поста в датафрейм

def condition(post_dictionary):
    habr_posts = pd.DataFrame()
    
    for i in range(len(post_dict)):
        for j in range(len(post_dict[i])):
            
            post_info = post_dictionary[i][j]
            
            if any([word in post_info['text'] for word in KEYWORDS]): 
                row = {'date': post_info['date'], 
                   'title': post_info['title'], 
                   'link': post_info['link']}
                
                habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])   
            
    return habr_posts

In [195]:
condition(post_dict)

,date,link,title
0,сегодня в 10:58,https://habr.com/ru/company/oleg-bunin/blog/51...,\n«Я что-то накодил и все упало»: провалы в Py...
0,10 августа 2020 в 16:16,https://habr.com/ru/company/ruvds/blog/514538/,"\nА вы знаете о том, что в Python есть встроен..."
0,9 августа 2020 в 21:40,https://habr.com/ru/post/514444/,"\nТри редко используемых возможности Python 3,..."
0,9 августа 2020 в 08:21,https://habr.com/ru/post/514384/,\nМониторинг демон на Asyncio + Dependency Inj...
0,сегодня в 10:58,https://habr.com/ru/company/oleg-bunin/blog/51...,\n«Я что-то накодил и все упало»: провалы в Py...
0,10 августа 2020 в 16:16,https://habr.com/ru/company/ruvds/blog/514538/,"\nА вы знаете о том, что в Python есть встроен..."
0,9 августа 2020 в 21:40,https://habr.com/ru/post/514444/,"\nТри редко используемых возможности Python 3,..."
0,9 августа 2020 в 08:21,https://habr.com/ru/post/514384/,\nМониторинг демон на Asyncio + Dependency Inj...
0,сегодня в 10:58,https://habr.com/ru/company/oleg-bunin/blog/51...,\n«Я что-то накодил и все упало»: провалы в Py...
0,10 августа 2020 в 16:16,https://habr.com/ru/company/ruvds/blog/514538/,"\nА вы знаете о том, что в Python есть встроен..."


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

##  Задание 2.

### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

In [199]:
import json
from datetime import datetime

In [197]:
emails = ['tischa.tascha@gmail.com', 'marina88@gmail.com']
leak_url = 'https://digibody.avast.com/v1/web/leaks' 

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [201]:
json_response = []
email_leaks = pd.DataFrame()

for i in range(len(emails)):
    
    # итерация по имейлам
    payload = {'email': emails[i]}  
    
    # привести инпут к json формату 
    r = requests.post(leak_url, json=payload)
    json_response = r.json()
    
    # берем только нужную нам инфу
    leaks = json_response['value']
        
    for leak in leaks:     
        leak_date =  datetime.fromtimestamp(int(str(leak['leak_date'])[:10])).strftime('%Y-%m-%d %H:%M:%S') 
        row = {'email': payload.values(), 'date': leak_date, 'source': leak['leak_info']['title'], 'description': leak['leak_info']['description']}    
        email_leaks = pd.concat([email_leaks, pd.DataFrame([row])]) 
        
print(email_leaks)

                  date                                        description  \
0  2019-02-06 01:00:00  On January 7, 2019, an online user named Sanix...   
0  2020-06-25 02:00:00  At an unconfirmed date, this "Cash Cloud" Comb...   
0  2019-06-13 02:00:00  In May 2019, graphic-design site Canva's datab...   
0  2018-08-24 02:00:00  In 2018, education tech company Chegg's databa...   
0  2019-01-29 01:00:00  On January 7, 2019, an online user named Sanix...   
0  2019-05-16 02:00:00  In June 2018, fast-fashion retailer Shein's da...   
0  2019-01-25 01:00:00  On January 7, 2019, an online user named Sanix...   
0  2017-12-22 01:00:00  The proliferation of stolen or leaked database...   
0  2018-08-24 02:00:00  This combolist was compiled from a variety of ...   
0  2019-02-06 01:00:00  On January 7, 2019, an online user named Sanix...   
0  2020-06-25 02:00:00  At an unconfirmed date, this "Cash Cloud" Comb...   
0  2017-10-09 02:00:00  The proliferation of stolen or leaked database...   

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.